## NYC Restaurant Inspections 
### Author: Jack Robbins

**Dataset Used**: https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/about_data

In [69]:
# Important imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [70]:
inspections = pd.read_csv("data/DOHMH_New_York_City_Restaurant_Inspection_Results_20241121.csv", low_memory=False)

In [71]:
inspections

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50145878,ANOTHER COUNTRY,Manhattan,10,EAST 16 STREET,10003.0,9175411321,NaN,01/01/1900,NaN,...,NaN,40.737151,-73.992263,105.0,2.0,5200.0,1016078.0,1.008430e+09,MN13,NaN
1,50148522,OK CANAAN,Queens,4318,MAIN ST,11355.0,7188868844,NaN,01/01/1900,NaN,...,NaN,40.751984,-73.826484,407.0,20.0,79702.0,4115474.0,4.051258e+09,QN22,NaN
2,50118771,DELI PIZZA LUNCHEONETTE,Brooklyn,603,AVENUE Z,11223.0,3472071540,Pizza,02/23/2022,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Initial Inspection,40.586061,-73.971655,313.0,47.0,37402.0,3195689.0,3.072130e+09,BK26,NaN
3,50161894,PATOK,Manhattan,104,W 35TH ST,10001.0,9173276332,NaN,01/01/1900,NaN,...,NaN,40.750616,-73.987711,105.0,4.0,10900.0,NaN,1.000000e+00,MN17,NaN
4,50114783,"HUDSON NIA JFK T1, JV",Queens,JFK,TERMINAL 1,11430.0,201 8218189,NaN,01/01/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262409,50016265,ADORO LEI,Manhattan,287,HUDSON STREET,10013.0,6466665096,Pizza,10/31/2024,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.725439,-74.007638,102.0,3.0,3700.0,1010311.0,1.005940e+09,MN24,NaN
262410,50091187,Luv Pizza,Manhattan,485,7 AVENUE,10018.0,6466924282,Pizza,04/13/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.752458,-73.989544,105.0,3.0,10900.0,1015235.0,1.008128e+09,MN17,NaN
262411,50136479,CECCHI'S BAR & GRILL,Manhattan,105,WEST 13 STREET,10011.0,9172501151,American,06/02/2023,Violations were cited in the following area(s).,...,Pre-permit (Non-operational) / Initial Inspection,40.736882,-73.997842,102.0,3.0,7100.0,1010653.0,1.006090e+09,MN23,NaN
262412,50095003,TWO BROTHERS PIZZA,Bronx,3039,BUHRE AVENUE,10461.0,7188246261,Pizza,11/04/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.847437,-73.831102,210.0,13.0,26602.0,2046599.0,2.041960e+09,BX10,NaN


In [72]:
# Let's get an idea of the shape of the graph
inspections.shape

(262414, 27)

In [73]:
null_values = inspections.isnull().sum()
print(null_values)

CAMIS                         0
DBA                           0
BORO                          0
BUILDING                    375
STREET                        3
ZIPCODE                    2684
PHONE                         3
CUISINE DESCRIPTION        3221
INSPECTION DATE               0
ACTION                     3221
VIOLATION CODE             4838
VIOLATION DESCRIPTION      4838
CRITICAL FLAG                 0
SCORE                     13310
GRADE                    136589
GRADE DATE               145829
RECORD DATE                   0
INSPECTION TYPE            3221
Latitude                    360
Longitude                   360
Community Board            3326
Council District           3311
Census Tract               3311
BIN                        4656
BBL                         645
NTA                        3326
Location Point1          262414
dtype: int64


### Let's drop unneeded columns

The columns BIN, BBL, NTA, and location point 1 have no official description on the data page and are therefore useless to us. We'll get rid of them. We'll also remove the CAMIS, GRADE DATE, PHONE, DBA, VIOLATION DESCRIPTION, latitude and longitude columns. These columns are documented but they are not useful to us, so it makes sense to remove.

In [74]:
inspections.drop(['Location Point1', 'NTA', 'BBL', 'BIN', 'CAMIS', 'GRADE DATE', 'PHONE', 'Latitude', 'Longitude', 'DBA',\
                  'BUILDING', 'STREET', 'VIOLATION DESCRIPTION', 'RECORD DATE'], axis = 1, inplace=True)
null_values = inspections.isnull().sum()
print(null_values)

BORO                        0
ZIPCODE                  2684
CUISINE DESCRIPTION      3221
INSPECTION DATE             0
ACTION                   3221
VIOLATION CODE           4838
CRITICAL FLAG               0
SCORE                   13310
GRADE                  136589
INSPECTION TYPE          3221
Community Board          3326
Council District         3311
Census Tract             3311
dtype: int64


### Let's analyze these findings 
As we can see above there are a lot of null values for the grade and grade date. The grade date would be very interesting for us to look at, so it's tempting to try and either fill those nulls or drop those columns

In [75]:
inspections['SCORE'].describe()

count    249104.000000
mean         24.115743
std          18.176170
min           0.000000
25%          12.000000
50%          20.000000
75%          32.000000
max         168.000000
Name: SCORE, dtype: float64

In [76]:
# Let's remove all rows that have a null score
inspections.dropna(subset=['SCORE'], inplace=True)

### Filling in missing grades

From the [NYC department of health](https://www.nyc.gov/assets/doh/downloads/pdf/about/healthcode/health-code-chapter23.pdf) website, the letter grade based off of score is as follows
* Grade A: 0-13 points scored
* Grade B: 14-27 points scored
* Grade C: >=28 points scored

We can use this to figure out what the grades are now

In [77]:
def grade_from_score(score):
    if score < 14:
        return 'A'
    elif score < 28:
        return 'B'
    else:
        return 'C'
    
# Fill in the grade based on score
for index, row in inspections.iterrows():
    inspections.at[index, 'GRADE'] = grade_from_score(int(row['SCORE']))

In [78]:
null_values = inspections.isnull().sum()
print(null_values)

BORO                      0
ZIPCODE                2523
CUISINE DESCRIPTION       0
INSPECTION DATE           0
ACTION                    0
VIOLATION CODE         1020
CRITICAL FLAG             0
SCORE                     0
GRADE                     0
INSPECTION TYPE           0
Community Board        3094
Council District       3079
Census Tract           3079
dtype: int64


In [79]:
inspections.shape

(249104, 13)

## Handling the rest of the nulls

At this point, we have a small number of rows that still contain null values. This is an acceptable loss for us, so we will go through and drop them.

In [80]:
inspections.dropna(how='any', inplace=True)

In [81]:
# Let's see how we did
null_values = inspections.isnull().sum()
print(null_values)

BORO                   0
ZIPCODE                0
CUISINE DESCRIPTION    0
INSPECTION DATE        0
ACTION                 0
VIOLATION CODE         0
CRITICAL FLAG          0
SCORE                  0
GRADE                  0
INSPECTION TYPE        0
Community Board        0
Council District       0
Census Tract           0
dtype: int64


In [82]:
inspections.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245038 entries, 2 to 262413
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   BORO                 245038 non-null  object 
 1   ZIPCODE              245038 non-null  float64
 2   CUISINE DESCRIPTION  245038 non-null  object 
 3   INSPECTION DATE      245038 non-null  object 
 4   ACTION               245038 non-null  object 
 5   VIOLATION CODE       245038 non-null  object 
 6   CRITICAL FLAG        245038 non-null  object 
 7   SCORE                245038 non-null  float64
 8   GRADE                245038 non-null  object 
 9   INSPECTION TYPE      245038 non-null  object 
 10  Community Board      245038 non-null  float64
 11  Council District     245038 non-null  float64
 12  Census Tract         245038 non-null  float64
dtypes: float64(5), object(8)
memory usage: 26.2+ MB


## Remaining cleanup

Even though we no longer have any null values in our dataframe, that does not mean that there isn't still junk in there. Let's look at each feature and see what still needs to be cleaned up

In [88]:
# Boro looks good to me
inspections['BORO'].unique()

array(['Brooklyn', 'Manhattan', 'Bronx', 'Queens', 'Staten Island'],
      dtype=object)

In [84]:
inspections['CUISINE DESCRIPTION'].unique()

array(['Pizza', 'Seafood', 'American', 'Creole/Cajun', 'Sandwiches',
       'Mexican', 'Jewish/Kosher', 'Spanish', 'Latin American',
       'Bakery Products/Desserts', 'Chinese', 'Other', 'Caribbean',
       'Thai', 'Creole', 'Mediterranean', 'Italian', 'Filipino',
       'Tex-Mex', 'Salads', 'Korean', 'Pancakes/Waffles',
       'Frozen Desserts', 'Japanese', 'Portuguese', 'Russian', 'Irish',
       'Coffee/Tea', 'Bangladeshi', 'Indian', 'Hamburgers', 'Tapas',
       'Chicken', 'Asian/Asian Fusion', 'Indonesian', 'Greek', 'French',
       'German', 'Southeast Asian', 'Donuts', 'Soul Food',
       'Eastern European', 'Fusion', 'Chinese/Cuban', 'Vegan',
       'Middle Eastern', 'Vegetarian', 'Pakistani', 'Peruvian', 'Polish',
       'Bagels/Pretzels', 'Sandwiches/Salads/Mixed Buffet',
       'Juice, Smoothies, Fruit Salads', 'Chinese/Japanese', 'Steakhouse',
       'Bottled Beverages', 'Barbecue', 'African', 'Nuts/Confectionary',
       'English', 'Armenian', 'New American', 'Turkish', '

In [86]:
# How many of these do we have?
inspections[inspections['CUISINE DESCRIPTION'] == 'Not Listed/Not Applicable']

,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,CRITICAL FLAG,SCORE,GRADE,INSPECTION TYPE,Community Board,Council District,Census Tract
2296,Brooklyn,11201.0,Not Listed/Not Applicable,06/24/2024,Violations were cited in the following area(s).,10G,Not Critical,25.0,B,Cycle Inspection / Initial Inspection,302.0,33.0,1500.0
3485,Brooklyn,11211.0,Not Listed/Not Applicable,12/14/2022,Violations were cited in the following area(s).,04K,Critical,11.0,A,Cycle Inspection / Initial Inspection,301.0,34.0,52300.0
5465,Queens,11101.0,Not Listed/Not Applicable,12/02/2021,Violations were cited in the following area(s).,04A,Critical,12.0,A,Pre-permit (Operational) / Initial Inspection,402.0,26.0,700.0
9358,Manhattan,10012.0,Not Listed/Not Applicable,07/11/2024,Violations were cited in the following area(s).,10F,Not Critical,32.0,C,Cycle Inspection / Initial Inspection,102.0,2.0,6500.0
10333,Brooklyn,11201.0,Not Listed/Not Applicable,12/19/2022,Violations were cited in the following area(s).,10F,Not Critical,3.0,A,Pre-permit (Operational) / Second Compliance I...,302.0,33.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244034,Brooklyn,11201.0,Not Listed/Not Applicable,02/22/2022,Violations were cited in the following area(s).,04N,Critical,0.0,A,Pre-permit (Operational) / Initial Inspection,302.0,33.0,1500.0
247519,Brooklyn,11201.0,Not Listed/Not Applicable,02/22/2022,Violations were cited in the following area(s).,10H,Not Critical,0.0,A,Pre-permit (Operational) / Initial Inspection,302.0,33.0,1500.0
250994,Queens,11101.0,Not Listed/Not Applicable,05/21/2024,Violations were cited in the following area(s).,10F,Not Critical,9.0,A,Cycle Inspection / Initial Inspection,402.0,26.0,700.0
251180,Manhattan,10002.0,Not Listed/Not Applicable,10/20/2022,Violations were cited in the following area(s).,10F,Not Critical,13.0,A,Pre-permit (Operational) / Initial Inspection,103.0,1.0,1800.0
